# Effect of including average bonus + save + card points

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

from airsenal.framework.prediction_utils import (
    fit_bonus_points,
    fit_card_points,
    fit_save_points,
)
from airsenal.framework.schema import Player, get_connection_string, session
from airsenal.framework.season import CURRENT_SEASON
from airsenal.framework.utils import get_player, list_players

engine = create_engine(get_connection_string())

In [69]:
df_bonus = fit_bonus_points()[0]
df_cards = fit_card_points()
df_saves = fit_save_points()

df = pd.DataFrame({"bonus": df_bonus, "cards": df_cards, "saves": df_saves})

df.fillna(0, inplace=True)
df["TOTAL"] = df.sum(axis=1)

players = pd.read_sql(session.query(Player).statement, engine)

df = pd.merge(df, players, how="left", left_on="player_id", right_on="player_id")

current_players = list_players()
current_ids = [p.player_id for p in current_players]
current_positions = pd.Series(
    {p.player_id: p.position(CURRENT_SEASON) for p in current_players}, name="position"
)
df = df[df.player_id.isin(current_ids)]
df = pd.merge(df, current_positions, how="left", left_on="player_id", right_index=True)
df.set_index(["player_id", "name"], inplace=True)

## Gain most overall

In [71]:
df.sort_values(by="TOTAL", ascending=False).head(20)

bonus     cards     saves  \
player_id name                                                          
325       Karl Darlow                    0.384615  0.000000  1.076923   
128       Vicente Guaita                 0.473684 -0.051724  0.807018   
431       Lukasz Fabianski               0.333333 -0.009615  0.892157   
96        Nick Pope                      0.567568 -0.053333  0.689189   
483       Aaron Ramsdale                 0.325000  0.000000  0.850000   
126       Wayne Hennessey                0.361702 -0.020833  0.787234   
302       Bruno Miguel Borges Fernandes  1.312500 -0.187500  0.000000   
523       Fraser Forster                 0.285714  0.000000  0.809524   
393       Paulo Gazzaniga                0.363636 -0.045455  0.761905   
8         Bernd Leno                     0.269841 -0.030769  0.825397   
433       Roberto Jimenez Gago           0.100000  0.000000  0.900000   
305       Dean Henderson                 0.444444 -0.055556  0.611111   
315       Mason Greenwood                1.000000  0.000000  0.000000   
371       Angus Gunn                     0.318182  0.000000  0.681818   
482       Joe Hart                       0.263158  0.000000  0.736842   
70        Mathew Ryan                    0.336283 -0.017699  0.672566   
383       Hugo Lloris                    0.315217 -0.010753  0.663043   
326       Martin Dubravka                0.250000 -0.011364  0.693182   
366       Danny Ings                     0.928571 -0.054795  0.000000   
4         Pierre-Emerick Aubameyang      0.937500 -0.079545  0.000000   

                                            TOTAL position  
player_id name                                              
325       Karl Darlow                    1.461538       GK  
128       Vicente Guaita                 1.228978       GK  
431       Lukasz Fabianski               1.215875       GK  
96        Nick Pope                      1.203423       GK  
483       Aaron Ramsdale                 1.175000       GK  
126       Wayne Hennessey                1.128103       GK  
302       Bruno Miguel Borges Fernandes  1.125000      MID  
523       Fraser Forster                 1.095238       GK  
393       Paulo Gazzaniga                1.080087       GK  
8         Bernd Leno                     1.064469       GK  
433       Roberto Jimenez Gago           1.000000       GK  
305       Dean Henderson                 1.000000       GK  
315       Mason Greenwood                1.000000      MID  
371       Angus Gunn                     1.000000       GK  
482       Joe Hart                       1.000000       GK  
70        Mathew Ryan                    0.991150       GK  
383       Hugo Lloris                    0.967508       GK  
326       Martin Dubravka                0.931818       GK  
366       Danny Ings                     0.873777      FWD  
4         Pierre-Emerick Aubameyang      0.857955      MID

## Lose most overall

In [77]:
df.sort_values(by="TOTAL", ascending=True).head(20)

bonus     cards  saves     TOTAL  \
player_id name                                                             
172       Kevin McDonald             0.000000 -0.466667    0.0 -0.466667   
227       Daniel Amartey             0.000000 -0.333333    0.0 -0.333333   
174       Stefan Johansen            0.000000 -0.333333    0.0 -0.333333   
82        Phil Bardsley              0.113208 -0.400000    0.0 -0.286792   
77        Tariq Lamptey              0.000000 -0.250000    0.0 -0.250000   
131       James McCarthy             0.000000 -0.243902    0.0 -0.243902   
364       Oriol Romeu Vidal          0.053333 -0.285714    0.0 -0.232381   
76        Yves Bissouma              0.000000 -0.230769    0.0 -0.230769   
408       Hal Robson-Kanu            0.000000 -0.217391    0.0 -0.217391   
25        Matteo Guendouzi           0.051282 -0.263158    0.0 -0.211876   
456       Ryan Bennett               0.069767 -0.280000    0.0 -0.210233   
64        Florin Andone              0.100000 -0.307692    0.0 -0.207692   
378       Yan Valery                 0.000000 -0.205882    0.0 -0.205882   
412       Kyle Bartley               0.000000 -0.200000    0.0 -0.200000   
2         Sokratis Papastathopoulos  0.209302 -0.409091    0.0 -0.199789   
477       Luke Freeman               0.000000 -0.181818    0.0 -0.181818   
173       Maxime Le Marchand         0.041667 -0.222222    0.0 -0.180556   
396       Pierre-Emile Højbjerg      0.108434 -0.277778    0.0 -0.169344   
384       Danny Rose                 0.102041 -0.271186    0.0 -0.169146   
403       Giovani Lo Celso           0.066667 -0.233333    0.0 -0.166667   

                                    position  
player_id name                                
172       Kevin McDonald                 MID  
227       Daniel Amartey                 DEF  
174       Stefan Johansen                MID  
82        Phil Bardsley                  DEF  
77        Tariq Lamptey                  DEF  
131       James McCarthy                 MID  
364       Oriol Romeu Vidal              MID  
76        Yves Bissouma                  MID  
408       Hal Robson-Kanu                FWD  
25        Matteo Guendouzi               MID  
456       Ryan Bennett                   DEF  
64        Florin Andone                  FWD  
378       Yan Valery                     DEF  
412       Kyle Bartley                   DEF  
2         Sokratis Papastathopoulos      DEF  
477       Luke Freeman                   MID  
173       Maxime Le Marchand             DEF  
396       Pierre-Emile Højbjerg          MID  
384       Danny Rose                     DEF  
403       Giovani Lo Celso               MID

## Top GK

In [72]:
df[df.position == "GK"].sort_values(by="TOTAL", ascending=False).head(20)

bonus     cards     saves     TOTAL  \
player_id name                                                           
325       Karl Darlow           0.384615  0.000000  1.076923  1.461538   
128       Vicente Guaita        0.473684 -0.051724  0.807018  1.228978   
431       Lukasz Fabianski      0.333333 -0.009615  0.892157  1.215875   
96        Nick Pope             0.567568 -0.053333  0.689189  1.203423   
483       Aaron Ramsdale        0.325000  0.000000  0.850000  1.175000   
126       Wayne Hennessey       0.361702 -0.020833  0.787234  1.128103   
523       Fraser Forster        0.285714  0.000000  0.809524  1.095238   
393       Paulo Gazzaniga       0.363636 -0.045455  0.761905  1.080087   
8         Bernd Leno            0.269841 -0.030769  0.825397  1.064469   
433       Roberto Jimenez Gago  0.100000  0.000000  0.900000  1.000000   
482       Joe Hart              0.263158  0.000000  0.736842  1.000000   
371       Angus Gunn            0.318182  0.000000  0.681818  1.000000   
305       Dean Henderson        0.444444 -0.055556  0.611111  1.000000   
70        Mathew Ryan           0.336283 -0.017699  0.672566  0.991150   
383       Hugo Lloris           0.315217 -0.010753  0.663043  0.967508   
326       Martin Dubravka       0.250000 -0.011364  0.693182  0.931818   
157       Jordan Pickford       0.299145 -0.025641  0.572650  0.846154   
12        Emiliano Martínez     0.300000 -0.181818  0.700000  0.818182   
151       Jonas Lössl           0.289855 -0.028986  0.544118  0.804987   
28        Tom Heaton            0.166667 -0.069767  0.707317  0.804216   

                               position  
player_id name                           
325       Karl Darlow                GK  
128       Vicente Guaita             GK  
431       Lukasz Fabianski           GK  
96        Nick Pope                  GK  
483       Aaron Ramsdale             GK  
126       Wayne Hennessey            GK  
523       Fraser Forster             GK  
393       Paulo Gazzaniga            GK  
8         Bernd Leno                 GK  
433       Roberto Jimenez Gago       GK  
482       Joe Hart                   GK  
371       Angus Gunn                 GK  
305       Dean Henderson             GK  
70        Mathew Ryan                GK  
383       Hugo Lloris                GK  
326       Martin Dubravka            GK  
157       Jordan Pickford            GK  
12        Emiliano Martínez          GK  
151       Jonas Lössl                GK  
28        Tom Heaton                 GK

## Top DEF

In [73]:
df[df.position == "DEF"].sort_values(by="TOTAL", ascending=False).head(20)

bonus     cards  saves  \
player_id name                                                          
259       Trent Alexander-Arnold            0.759036 -0.134831    0.0   
255       Andrew Robertson                  0.652632 -0.082474    0.0   
297       Phil Jones                        0.628571 -0.069767    0.0   
29        Ahmed El Mohamady                 0.600000 -0.055556    0.0   
127       Mamadou Sakho                     0.563636 -0.112903    0.0   
102       César Azpilicueta                 0.468468 -0.053097    0.0   
155       Lucas Digne                       0.600000 -0.191781    0.0   
498       Timothy Castagne                  0.400000  0.000000    0.0   
123       Reece James                       0.473684 -0.074074    0.0   
226       Ricardo Domingos Barbosa Pereira  0.523810 -0.126984    0.0   
104       Marcos Alonso                     0.587500 -0.200000    0.0   
250       Virgil van Dijk                   0.407767 -0.028571    0.0   
269       Nicolás Otamendi                  0.500000 -0.171053    0.0   
457       Matt Doherty                      0.409091 -0.090909    0.0   
121       Fikayo Tomori                     0.500000 -0.187500    0.0   
139       Patrick van Aanholt               0.418605 -0.107527    0.0   
240       Luke Thomas                       0.300000  0.000000    0.0   
494       Gabriel Magalhães                 0.300000  0.000000    0.0   
156       Michael Keane                     0.380435 -0.082474    0.0   
218       Christian Fuchs                   0.322581 -0.025000    0.0   

                                               TOTAL position  
player_id name                                                 
259       Trent Alexander-Arnold            0.624205      DEF  
255       Andrew Robertson                  0.570157      DEF  
297       Phil Jones                        0.558804      DEF  
29        Ahmed El Mohamady                 0.544444      DEF  
127       Mamadou Sakho                     0.450733      DEF  
102       César Azpilicueta                 0.415371      DEF  
155       Lucas Digne                       0.408219      DEF  
498       Timothy Castagne                  0.400000      DEF  
123       Reece James                       0.399610      DEF  
226       Ricardo Domingos Barbosa Pereira  0.396825      DEF  
104       Marcos Alonso                     0.387500      DEF  
250       Virgil van Dijk                   0.379196      DEF  
269       Nicolás Otamendi                  0.328947      DEF  
457       Matt Doherty                      0.318182      DEF  
121       Fikayo Tomori                     0.312500      DEF  
139       Patrick van Aanholt               0.311078      DEF  
240       Luke Thomas                       0.300000      DEF  
494       Gabriel Magalhães                 0.300000      DEF  
156       Michael Keane                     0.297961      DEF  
218       Christian Fuchs                   0.297581      DEF

## Top MID

In [74]:
df[df.position == "MID"].sort_values(by="TOTAL", ascending=False).head(20)

bonus     cards  saves  \
player_id name                                                                 
302       Bruno Miguel Borges Fernandes            1.312500 -0.187500    0.0   
315       Mason Greenwood                          1.000000  0.000000    0.0   
4         Pierre-Emerick Aubameyang                0.937500 -0.079545    0.0   
284       Phil Foden                               0.818182  0.000000    0.0   
272       Kevin De Bruyne                          0.873418 -0.075269    0.0   
275       Riyad Mahrez                             0.784615 -0.020619    0.0   
436       Andriy Yarmolenko                        0.833333 -0.147059    0.0   
254       Mohamed Salah                            0.669811 -0.027027    0.0   
390       Heung-Min Son                            0.670886 -0.108911    0.0   
478       Willian Borges Da Silva                  0.613333 -0.065421    0.0   
251       Sadio Mané                               0.663158 -0.116505    0.0   
276       Raheem Sterling                          0.681319 -0.137255    0.0   
306       Marcus Rashford                          0.644737 -0.118812    0.0   
57        Pascal Groß                              0.597222 -0.074468    0.0   
106       Ross Barkley                             0.454545 -0.019231    0.0   
26        Gabriel Teodoro Martinelli Silva         0.500000 -0.071429    0.0   
507       Ryan Fraser                              0.468354 -0.043011    0.0   
281       Bernardo Mota Veiga de Carvalho e Silva  0.492308 -0.076190    0.0   
229       Ayoze Pérez                              0.441860 -0.046296    0.0   
33        Conor Hourihane                          0.529412 -0.137931    0.0   

                                                      TOTAL position  
player_id name                                                        
302       Bruno Miguel Borges Fernandes            1.125000      MID  
315       Mason Greenwood                          1.000000      MID  
4         Pierre-Emerick Aubameyang                0.857955      MID  
284       Phil Foden                               0.818182      MID  
272       Kevin De Bruyne                          0.798149      MID  
275       Riyad Mahrez                             0.763997      MID  
436       Andriy Yarmolenko                        0.686275      MID  
254       Mohamed Salah                            0.642784      MID  
390       Heung-Min Son                            0.561975      MID  
478       Willian Borges Da Silva                  0.547913      MID  
251       Sadio Mané                               0.546653      MID  
276       Raheem Sterling                          0.544064      MID  
306       Marcus Rashford                          0.525925      MID  
57        Pascal Groß                              0.522754      MID  
106       Ross Barkley                             0.435315      MID  
26        Gabriel Teodoro Martinelli Silva         0.428571      MID  
507       Ryan Fraser                              0.425344      MID  
281       Bernardo Mota Veiga de Carvalho e Silva  0.416117      MID  
229       Ayoze Pérez                              0.395564      MID  
33        Conor Hourihane                          0.391481      MID

## Top FWD

In [75]:
df[df.position == "FWD"].sort_values(by="TOTAL", ascending=False).head(20)

bonus     cards  saves     TOTAL  \
player_id name                                                             
366       Danny Ings                 0.928571 -0.054795    0.0  0.873777   
268       Sergio Agüero              0.909091 -0.085366    0.0  0.823725   
460       Raúl Jiménez               0.891892 -0.088608    0.0  0.803284   
224       Jamie Vardy                0.902913 -0.110092    0.0  0.792821   
256       Divock Origi               0.800000 -0.024390    0.0  0.775610   
388       Harry Kane                 0.903226 -0.144330    0.0  0.758896   
282       Gabriel Fernando de Jesus  0.863636 -0.118280    0.0  0.745357   
68        Neal Maupay                0.781250 -0.075000    0.0  0.706250   
249       Roberto Firmino            0.707071 -0.017857    0.0  0.689214   
91        Chris Wood                 0.710526 -0.041667    0.0  0.668860   
118       Tammy Abraham              0.682927 -0.029851    0.0  0.653076   
78        Aaron Connolly             0.538462  0.000000    0.0  0.538462   
202       Patrick Bamford            0.500000  0.000000    0.0  0.500000   
169       Moise Kean                 0.500000 -0.032258    0.0  0.467742   
359       Lys Mousset                0.500000 -0.038961    0.0  0.461039   
108       Michy Batshuayi            0.454545  0.000000    0.0  0.454545   
343       Billy Sharp                0.600000 -0.148148    0.0  0.451852   
441       Sébastien Haller           0.478261 -0.028571    0.0  0.449689   
303       Anthony Martial            0.492537 -0.043956    0.0  0.448581   
6         Alexandre Lacazette        0.541667 -0.110000    0.0  0.431667   

                                    position  
player_id name                                
366       Danny Ings                     FWD  
268       Sergio Agüero                  FWD  
460       Raúl Jiménez                   FWD  
224       Jamie Vardy                    FWD  
256       Divock Origi                   FWD  
388       Harry Kane                     FWD  
282       Gabriel Fernando de Jesus      FWD  
68        Neal Maupay                    FWD  
249       Roberto Firmino                FWD  
91        Chris Wood                     FWD  
118       Tammy Abraham                  FWD  
78        Aaron Connolly                 FWD  
202       Patrick Bamford                FWD  
169       Moise Kean                     FWD  
359       Lys Mousset                    FWD  
108       Michy Batshuayi                FWD  
343       Billy Sharp                    FWD  
441       Sébastien Haller               FWD  
303       Anthony Martial                FWD  
6         Alexandre Lacazette            FWD

## Specific player

In [79]:
p = get_player("Fabinho").player_id
df[df.index.get_level_values(0) == p]

bonus    cards  saves     TOTAL position
player_id name                                                               
253       Fabio Henrique Tavares  0.177778 -0.20339    0.0 -0.025612      MID